In [ ]:
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import string
import re
from google.colab import drive 
import numpy as np
import pandas as pd
import plotly.express as px
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import operator
from copy import deepcopy
import xgboost
from sklearn.model_selection import KFold

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/Shared drives/Capstone Team 15 Drive/Work/')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Mounted at /content/drive


In [ ]:
X_post = pd.read_csv('after_CV_Doc2Vec_X-values.csv')
y_post = pd.read_csv('after_CV_Doc2Vec_y-values.csv')

In [ ]:
y_post

,Unnamed: 0,0
0,0,Laptop
1,1,VDI / DDV
2,2,IP Router
3,3,Business Application
4,4,Laptop
...,...,...
20139,20139,Software
20140,20140,VDI / DDV
20141,20141,Business Application
20142,20142,Active Directory Service


In [ ]:
X_post = X_post.drop(['Unnamed: 0'], axis=1)
y_post = y_post.drop(['Unnamed: 0'], axis=1)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics

model = xgboost.XGBClassifier(objective='multi:softmax')

rn = list(X_post.index)+ [1] * len(X_post.index)
estimators = [50,100,150,200,300]
kf16 = KFold(n_splits=16, shuffle=False)

def XGB_16CV_acc_n_estimators(j):
  acc = 0
  rn = list(X_post.index)+ [1] * len(X_post.index)
  kf16 = KFold(n_splits=16, shuffle=False)
  for train_index, test_index in kf16.split(rn):
    X_train = X_post[X_post.index.isin(train_index)]
    X_test = X_post[X_post.index.isin(test_index)]
    y_train = y_post[y_post.index.isin(train_index)]
    y_test = y_post[y_post.index.isin(test_index)]

    # train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize_text(r['comments_and_work_notes']), tags=[r.u_ci_class_name]), axis=1)
    # val_tagged = validation.apply(lambda r: TaggedDocument(words=tokenize_text(r['comments_and_work_notes']), tags=[r.u_ci_class_name]), axis=1)
    # model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
    # model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

    # #%%time
    # for epoch in range(30):
    #   model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    #   model_dbow.alpha -= 0.002
    #   model_dbow.min_alpha = model_dbow.alpha
    clf = xgboost.XGBClassifier(n_estimators = j, objective="multi:softmax") #multi:softmax
    clf.fit(X_train, y_train.values.ravel())
    ypred = clf.predict(X_test)
    acc += metrics.accuracy_score(y_test, ypred)

  avg_acc = acc / 16

  return avg_acc

best_n_estimators = 0
best_avg_acc = 0
for i in estimators:
  if XGB_16CV_acc_n_estimators(i) > best_avg_acc:
    best_avg_acc = XGB_16CV_acc_n_estimators(i)
    best_n_estimators = i

best_n_estimators
  


KeyboardInterrupt: ignored

In [ ]:
clf = xgboost.XGBClassifier(n_estimators = best_n_estimators, objective="multi:softmax") #multi:softmax
clf.fit(X_train, y_train.values.ravel())
ypred = clf.predict(X_test)
overall_acc = metrics.accuracy_score(y_test, ypred)

overall_acc

array([['Laptop'],
       ['VDI / DDV'],
       ['IP Router'],
       ...,
       ['Business Application'],
       ['Active Directory Service'],
       ['Computer']], dtype=object)

In [ ]:
while True:pass